In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime

In [21]:
site = 'https://www.oddsshark.com'
lg_tks = {}
lg_futs = {}
lg_tks['UFC'] = requests.get(
    'https://io.oddsshark.com/ticker/ufc', 
    headers = {
        'referer': site
    }
)

for league, active in lg_tks['UFC'].json()['leagues'].items():
    lg_futs[league.upper()] = BeautifulSoup(requests.get(f"{site}/{league}/odds/futures").content, 'html.parser')
    if active==True:
        lg_tks[league.upper()] = requests.get(
            'https://io.oddsshark.com/ticker/' + league, 
            headers = {
                'referer': site
            }
        ).json()['matchups']

In [ ]:
yesterday = date.today() - timedelta(days=1)
today = date.today()
tomorrow = date.today() + timedelta(days=1) 

In [26]:
lg_futs['NHL']

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html class="os-theme-dark" dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:video="http://ogp.me/ns/video#"><head profile="http://www.w3.org/1999/xhtml/vocab"><meta charset="utf-8"/> <!--[if IE]><![endif]--><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="application/ld+json">[{"@type": "SportsEvent", "@context": "http://schema.org", "inLanguage": "en-US", "name": "Montreal Canadiens vs Calgary Flames", "url": "https://www.oddsshark.com/nhl/calgary-montreal-odds-april-16-2021-1369831", "location": {
"@type": "Place",
"name": "Bell Centre",
"address": {
"@type": "PostalAddress",
"addressLo

In [ ]:
def simple_odds(odds):
    for game in odds:
        if game['type'] == 'date':
            print('--', game['date']['fullday'], game['date']['month'], game['date']['day'])
        if game["type"] == "matchup":
            if game['status'].startswith('FINAL'):
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_score'], '| font=Courier')
                print('----', game['home_short_name'], game['home_score'], '| font=Courier')
            else:
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_odds'] if game['away_odds'].startswith('-') else '+' + game['away_odds'], '| font=Courier')
                print('----', game['home_short_name'], game['home_odds'] if game['home_odds'].startswith('-') else '+' + game['home_odds'], '| font=Courier')

In [ ]:
for sport, odds in lg_tks.items():
    if sport != 'UFC':
        print(sport)
        simple_odds(odds)
        print('---')

In [ ]:
lg_tks['UFC']

In [ ]:
print('UFC')
for fight in lg_tks['UFC']:
    if fight['type'] == 'event':
        print('--', fight['event'])
    if fight["type"] == 'matchup':
        if not fight['status']:
            print('----', fight['event_date'][11:])
            print('----', fight['away_name'], fight['away_odds'] if fight['away_odds'].startswith('-') else '+' + fight['away_odds'], '| font=Courier')
            print('----', fight['home_name'], fight['home_odds'] if fight['home_odds'].startswith('-') else '+' + fight['home_odds'], '| font=Courier')
        else:
            print('----', 'FINAL')
            print('----', fight['away_name'], fight['status'] if fight['away_name'] == fight['winner'] else '', '| font=Courier')
            print('----', fight['home_name'], fight['status'] if fight['home_name'] == fight['winner'] else '', '| font=Courier')

In [ ]:
print(soup.prettify())

In [ ]:
opening_odds = soup.find_all('div', class_='op-item op-future-item')
opening_list = []
for elem in opening_odds:
    opening_list.append(elem.get_text())

In [ ]:
current_odds = soup.find_all('div', class_='op-item op-future-item no-vegas')
current_list = []
for elem in current_odds:
    current_list.append(elem.get_text())

In [ ]:
left_column = soup.find('div', class_='op-team-data-wrapper futures')
left_column = futures_teams.find_all(['div','span'], class_=['align-bottom','op-team hockey op-odd','op-team hockey op-even'])
team_list =[]
for elem in left_column:
    team_list.append(elem.get_text().strip())

In [4]:
def process_odds_section(soup, html_type, html_class):
    data = soup.find_all(html_type, class_=html_class)
    init_list = []
    daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    for elem in data:
        if not elem.get_text().startswith(tuple(daysofweek)):
            init_list.append(elem.get_text().strip())
    return init_list

In [29]:
# works for all leagues for current_odds
opening_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item ')
current_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item no-vegas')

In [17]:
team_list = process_odds_section(lg_futs['MLB'], ['div','span'], ['align-bottom','op-team baseball op-odd','op-team baseball op-even'])

In [55]:
future_odds_list = []
for a, b, c in zip(*[iter(current_odds)]*3): future_odds_list.append([a,b,c])

In [18]:
def create_futures_data(teams, opening, currents):
    
    futures_trios = []
    for a, b, c in zip(*[iter(currents)]*3):
        futures_trios.append([a,b,c])
    
    rows = []
    odds_inc = 0
    sports = ['MLB','NFL','NHL','NBA','College Basketball','College Football']
    for team in teams:
        if not team.startswith(tuple(sports)):
            rows.append([team,
                         opening[odds_inc],
                         futures_trios[odds_inc][0],
                         futures_trios[odds_inc][1],
                         futures_trios[odds_inc][2]])
            odds_inc += 1
        else:
            rows.append([team])
    return rows

In [19]:
all_odds = create_futures_data(team_list, opening_odds, current_odds)

IndexError: list index out of range

In [9]:
for odds in all_odds:
    if len(odds) > 1:
        print(odds[0], odds[1], odds[2], odds[3], odds[4])
    else:
        print(odds[0])

MLB - World Series Championship 2021
Arizona +6600 +20000 +20000 +20000
Atlanta +1100 +1200 +1400 +1400
Baltimore +8000 +10000 +10000 +10000
Boston +5000 +3500 +3500 +3500
Chi Cubs +2500 +5000 +5000 +5000
Chi White Sox +1200 +1200 +1200 +1200
Cincinnati +2000 +3500 +3300 +3300
Cleveland +1800 +5500 +4000 +4000
Colorado +8000 +25000 +25000 +25000
Detroit +8000 +15000 +15000 +15000
Houston +1600 +1600 +1400 +1400
Kansas City +10000 +8000 +8000 +8000
LA Angels +4000 +2200 +2200 +2200
LA Dodgers +400 +325 +300 +300
Miami +5000 +10000 +10000 +10000
Milwaukee +5000 +4000 +4000 +4000
Minnesota +1400 +1800 +1800 +1800
NY Mets +3000 +1200 +1200 +1200
NY Yankees +500 +650 +675 +675
Oakland +1800 +4000 +4000 +4000
Philadelphia +3300 +3000 +4000 +4000
Pittsburgh +25000 +40000 +40000 +40000
San Diego +900 +750 +800 +800
San Francisco +6600 +7000 +10000 +10000
Seattle +8000 +10000 +8000 +8000
St. Louis +3300 +2500 +2500 +2500
Tampa Bay +1100 +3300 +2800 +2800
Texas +10000 +17500 +17500 +17500
Toront

In [31]:
opening_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item ')
current_odds = process_odds_section(lg_futs['MLB'], 'div', 'op-item op-future-item no-vegas')
team_list = process_odds_section(lg_futs['MLB'], ['div','span'], ['align-bottom','op-team baseball op-odd','op-team baseball op-even'])
all_odds = create_futures_data(team_list, opening_odds, current_odds)
for odds in all_odds:
    if len(odds) > 1:
        print(odds[0], odds[1], odds[2], odds[3], odds[4])
    else:
        print(odds[0])

MLB - World Series Championship 2021
Arizona +6600 +20000 +20000 +20000
Atlanta +1100 +1200 +1400 +1400
Baltimore +8000 +10000 +10000 +10000
Boston +5000 +3000 +2800 +2800
Chi Cubs +2500 +5000 +5000 +5000
Chi White Sox +1200 +1200 +1200 +1200
Cincinnati +2000 +3500 +3300 +3300
Cleveland +1800 +5500 +4000 +4000
Colorado +8000 +25000 +25000 +25000
Detroit +8000 +15000 +12500 +12500
Houston +1600 +1600 +1600 +1600
Kansas City +10000 +8000 +8000 +8000
LA Angels +4000 +2500 +2200 +2200
LA Dodgers +400 +325 +300 +300
Miami +5000 +10000 +10000 +10000
Milwaukee +5000 +4000 +3500 +3500
Minnesota +1400 +1800 +1800 +1800
NY Mets +3000 +1000 +1200 +1200
NY Yankees +500 +650 +675 +675
Oakland +1800 +3300 +4000 +4000
Philadelphia +3300 +4000 +5000 +5000
Pittsburgh +25000 +40000 +30000 +30000
San Diego +900 +800 +800 +800
San Francisco +6600 +9000 +8000 +8000
Seattle +8000 +10000 +8000 +8000
St. Louis +3300 +2500 +2800 +2800
Tampa Bay +1100 +3500 +2800 +2800
Texas +10000 +17500 +15000 +15000
Toronto 

In [33]:
for sport in lg_futs:
    opening_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item ')
    current_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item no-vegas')
    team_list = process_odds_section(lg_futs[sport], ['div','span'], ['align-bottom',
                                                                      'op-team baseball op-odd',
                                                                      'op-team baseball op-even',
                                                                      'op-team basketball op-odd',
                                                                      'op-team basketball op-even',
                                                                      'op-team football op-odd',
                                                                      'op-team football op-even',
                                                                      'op-team hockey op-odd',
                                                                      'op-team hockey op-even'])
    all_odds = create_futures_data(team_list, opening_odds, current_odds)
    for odds in all_odds:
        if len(odds) > 1:
            print(odds[0], odds[1], odds[2], odds[3], odds[4])
        else:
            print(odds[0])

NHL - Stanley Cup 2020-21
Anaheim +8000 +150000 +75000 +75000
Arizona +4000 +8000 +7500 +7500
Boston +1200 +1600 +1400 +1400
Buffalo +8000 +150000 +300000 +300000
Calgary +3000 +15000 +5000 +5000
Carolina +2500 +1100 +1200 +1200
Chicago +8000 +10000 +6600 +6600
Colorado +700 +350 +700 +700
Columbus +5000 +40000 +8000 +8000
Dallas +2000 +6600 +6000 +6000
Detroit +20000 +150000 +150000 +150000
Edmonton +2500 +2000 +2200 +2200
Florida +5000 +2000 +1800 +1800
Los Angeles +8000 +20000 +6600 +6600
Minnesota +5000 +2000 +2000 +2000
Montreal +6000 +2200 +2800 +2800
Nashville +3000 +6000 +10000 +10000
New Jersey +5000 +100000 +30000 +30000
NY Islanders +2500 +1200 +1100 +1100
NY Rangers +2500 +7500 +6600 +6600
Ottawa +10000 +150000 +250000 +250000
Philadelphia +1600 +10000 +3300 +3300
Pittsburgh +2000 +2200 +2200 +2200
San Jose +6000 +15000 +25000 +25000
St. Louis +2000 +3000 +2500 +2500
Tampa Bay +700 +600 +600 +600
Toronto +2000 +700 +900 +900
Vancouver +2000 +15000 +10000 +10000
Vegas +700 +

NFL - Super Bowl LVI 2021-22
Arizona +4000 +3000 +3300 +3300
Atlanta +4000 +8000 +6600 +6600
Baltimore +1200 +1800 +1800 +1800
Buffalo +800 +1400 +1200 +1200
Carolina +5000 +6000 +5000 +5000
Chicago +6600 +6600 +5000 +5000
Cincinnati +8000 +15000 +8000 +8000
Cleveland +1600 +1800 +1600 +1600
Dallas +2500 +2800 +2800 +2800
Denver +6600 +6000 +5000 +5000
Detroit +10000 +15000 +15000 +15000
Green Bay +750 +1200 +1200 +1200
Houston +10000 +20000 +15000 +15000
Indianapolis +2500 +2500 +2000 +2000
Jacksonville +5000 +12500 +10000 +10000
Kansas City +550 +500 +550 +550
LA Chargers +3300 +3500 +3300 +3300
LA Rams +2500 +1300 +1200 +1200
Las Vegas +5000 +6600 +5000 +5000
Miami +2000 +3300 +3300 +3300
Minnesota +5000 +5000 +5000 +5000
New England +4000 +2800 +2800 +2800
New Orleans +1800 +2800 +2800 +2800
NY Giants +6600 +7000 +6600 +6600
NY Jets +10000 +10000 +6600 +6600
Philadelphia +5000 +8500 +6600 +6600
Pittsburgh +3000 +3300 +3300 +3300
San Francisco +1800 +1400 +1400 +1400
Seattle +1800 +

Yale +200000   
Youngstown State +100000   
